In [1]:
#from bs4 import BeautifulSoup
#import json
#import optparse
import os, regex as re
import pandas as pd

#import libraries 

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import re 
#import matplotlib.pyplot as plt 
#import seaborn as sns
import scipy as sp

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk import pos_tag

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2, SelectKBest

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#from sklearn.cross_validation import train_test_split, cross_val_score, cross_val_predict
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, cross_val_score

from sklearn import naive_bayes
from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from mlens.ensemble import SuperLearner

from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report

from nltk.stem.porter import PorterStemmer

import imblearn
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler

[MLENS] backend: threading


In [2]:
import pandas as pd
import re
import nltk

STOPWORDS = set(('i','im','ive', 'me','my','myself','we','our','ours','ourselves','you','youre','youve','youll','youd','your','yours','yourself','yourselves','he','him','his','himself','she','shes','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that','thatll','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','such','only','own','same','so','than','too','very','s','t','can','will','just','should','shouldve','now','d','ll','m','o','re','ve','y','ma'))

SNO = nltk.stem.SnowballStemmer('english')

def read_data(filename):
    reviews = []
    with open(filename, 'r') as fr:
        end_of_review = False
        for line in fr:
            line = re.sub(r'[:][\d]', " ", line)
            if "#label#:negative" in line:
                line = line.replace("#label#:negative", "")
                end_of_review = True
            elif "#label#:positive" in line:
                line = line.replace("#label#:positive", "")
                end_of_review = True
            
            if end_of_review:
                reviews.append(line.strip())
                end_of_review = False           
    return reviews

def process_data(path, sentiment):
    reviews_list = read_data(path + sentiment + '.review')
    df = pd.DataFrame({'reviews': reviews_list})
    df['label'] = 1 if sentiment == "positive" else 0
    df['#words'] = df['reviews'].apply(lambda x: len(x.split()))
    df['code'] = path.split("/")[-2]
    return df.sample(frac=1).reset_index(drop=True)

def main():
    base_path = 'Downloads/processed_acl/'
    categories = ['books', 'dvd', 'kitchen', 'electronics']
    df_list = []
    
    for category in categories:
        for sentiment in ['positive', 'negative']:
            df_list.append(process_data(base_path + category + '/', sentiment))

    # Example concatenation:
    df_books_dvd = pd.concat([df_list[0], df_list[2]], ignore_index=True)
    # You can continue with other combinations as needed...

if __name__ == "__main__":
    main()


<open file 'Downloads/processed_acl/books/negative.review', mode 'r+' at 0x11456e420>
<open file 'Downloads/processed_acl/books/positive.review', mode 'r+' at 0x11456e420>
<open file 'Downloads/processed_acl/dvd/negative.review', mode 'r+' at 0x11456e420>
<open file 'Downloads/processed_acl/dvd/positive.review', mode 'r+' at 0x11456e420>
<open file 'Downloads/processed_acl/kitchen/negative.review', mode 'r+' at 0x11456e420>
<open file 'Downloads/processed_acl/kitchen/positive.review', mode 'r+' at 0x11456e420>
<open file 'Downloads/processed_acl/electronics/negative.review', mode 'r+' at 0x11456e420>
<open file 'Downloads/processed_acl/electronics/positive.review', mode 'r+' at 0x11456e420>


In [9]:
sample_df = df_electronics
#Functions for preprocessing steps
stop = set(('i','im','ive', 'me','my','myself','we','our','ours','ourselves','you','youre','youve','youll','youd','your','yours','yourself','yourselves','he','him','his','himself','she','shes','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that','thatll','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','such','only','own','same','so','than','too','very','s','t','can','will','just','should','shouldve','now','d','ll','m','o','re','ve','y','ma'))
sno = nltk.stem.SnowballStemmer('english')

def replace_url(df,col,rm1,rm2):
    return(df[col].str.replace(rm1,rm2))

def extract_emo(df, col, emo):
    return(df[col].str.extractall(emo).unstack().apply(lambda x:' '.join(x.dropna()), axis=1))

def replace_emo(df,col,emo1,emo2):
    return(df[col].str.replace(emo1,emo2))

def replace_punct(df, col, punct1, punct2):
    return(df[col].str.replace(punct1, punct2))

def remove_numbers(df,col,rm1,rm2):
    return(df[col].str.replace(rm1,rm2))

def lower_words(df,col):
    return(df[col].apply(lambda x: " ".join(x.lower() for x in x.split())))

def remove_stop(df,col):
    return(df[col].apply(lambda x: " ".join(x for x in x.split() if x not in stop)))

def tokenize(df,col):
    return(df.apply(lambda row: nltk.word_tokenize(row[col]), axis=1))

def word_count(df,col):
    return(df[col].apply(lambda x: len(str(x).split(' '))))

def stemming(df,col):
    return(df[col].apply(lambda x: " ".join([sno.stem(word) for word in x.split()])))



def preprocess_data(df):
    df['nohtml'] = replace_url(df,'reviews','^http?:\/\/.*[\r\n]*','')
    df['nohtml'] = lower_words(df,'nohtml')
    df['nohtml'] = remove_numbers(df, 'nohtml', '[0-9]+',' ')
    df['nohtml'] = replace_punct(df, 'nohtml', '[^\w\s]',' ')
    df['nohtml'] = replace_punct(df, 'nohtml', '_',' ')
    df['nohtml'] = replace_punct(df, 'nohtml',r'\b(no|not|nt|dont|doesnt|doesn|don|didnt|cant|cannt|cannot|wouldnt|wont|couldnt|hasnt|havent|hadnt|shouldnt)\s+([a-z])',r'not \2')
    df['nohtml'] = remove_stop(df,'nohtml')
    df['tokenized'] = tokenize(df,'nohtml')
    df['#token'] = word_count(df,'tokenized')
    return df

# Preprocess the sample dataset
sample_df = preprocess_data(df_electronics)

# Split based on word count
tokens_thresholds = {
    'books': 80,
    'dvd': 75,
    'electronics': 55,
    'kitchen': 54
}

sub_datasets = {}
for category, threshold in tokens_thresholds.items():
    sub_datasets[category] = sample_df[sample_df["#token"] > threshold].reset_index(drop=True)

# Combine datasets
combinations = [
    ('books', 'dvd'),
    ('books', 'kitchen'),
    ('dvd', 'books'),
    ('electronics', 'books'),
    ('kitchen', 'books'),
    ('electronics', 'dvd'),
    ('kitchen', 'dvd'),
    ('books', 'electronics'),
    ('dvd', 'electronics'),
    ('kitchen', 'electronics'),
    ('electronics', 'kitchen'),
    ('dvd', 'kitchen')
]

combined_datasets = {}
for combo in combinations:
    key = f"{combo[0]}_{combo[1]}"
    combined_datasets[key] = sub_datasets[combo[0]].append(sub_datasets[combo[1]], ignore_index=True)

# If you want to get one of the combined datasets:
sample_df1 = combined_datasets['electronics_kitchen'].copy(deep=True)






In [14]:


# chi square for the important features per product category
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from nltk.tag import pos_tag
from fastText import load_model
import collections
import numpy as np

def extract_important_features(df, text_column, target_column, n_features=1000):
    count_vect = CountVectorizer()
    X = count_vect.fit_transform(df[text_column].values)
    feature_names = count_vect.get_feature_namessample_df = sample_df1.copy()()   
    chi2_scores = chi2(X, df[target_column])
    sorted_scores = sorted(zip(feature_names, chi2_scores[0]), key=lambda x:x[1])[-n_features:]
    return [k for k, v in sorted_scores if v < 0.05]





/Users/aikaterinikatsarou/homebrew_high_sierra/lib/python2.7/site-packages/ipykernel_launcher.py:56: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


# Exclude Nouns

In [15]:
def exclude_nouns(token_list):
    return [word for word, pos in pos_tag(token_list) if not pos.startswith('N')]


/Users/aikaterinikatsarou/homebrew_high_sierra/lib/python2.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


# Import FastText bin

In [16]:
def df_to_fasttext_data(df, token_column, model):
    all_words = set(w for tokens in df[token_column] for w in tokens)
    return {word: model.get_word_vector(word).astype('float32') for word in all_words}

# Load fastText model (uncomment the appropriate line)
# ft_model = FastText("Downloads/cc.en.300.bin")
# ft_model = fastText.load_model('Downloads/cc.en.300.bin')
ft_model = load_model('Downloads/cc.en.300.bin')

# Extract important features based on Chi-squared scores
cat_features = extract_important_features(sample_df, 'nohtml', 'code')
label_features = extract_important_features(sample_df, 'nohtml', 'label')

# Exclude nouns and create a new column 'tokenized2'
sample_df["tokenized2"] = sample_df["tokenized"].apply(exclude_nouns)

# Extract FastText vectors
fasttext_dict = df_to_fasttext_data(sample_df, 'tokenized', ft_model)
fasttext2_dict = df_to_fasttext_data(sample_df, 'tokenized2', ft_model)



In [17]:
fasttext = df_to_data(sample_df, X)
fasttext2 = df_to_data(sample_df, X1)

In [18]:
#use the pretrained fasttext


#%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
#from sklearn.cross_validation import cross_val_score
#from sklearn.cross_validation import StratifiedShuffleSplit
from collections import Counter, defaultdict

In [19]:

import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

class BaseEmbeddingVectorizer:
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(word2vec[next(iter(word2vec))]) if word2vec else None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        raise NotImplementedError()

    def get_params(self, deep=True):
        return {'word2vec': self.word2vec}


class MeanEmbeddingVectorizer(BaseEmbeddingVectorizer):
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


class TfidfEmbeddingVectorizer(BaseEmbeddingVectorizer):
    def fit(self, X, y=None):
        self.tfidf = TfidfVectorizer(analyzer=lambda x: x)
        self.tfidf.fit(X)
        max_idf = max(self.tfidf.idf_)
        self.word2weight = defaultdict(lambda: max_idf,
                                       [(w, self.tfidf.idf_[i]) for w, i in self.tfidf.vocabulary_.items()])
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] * self.word2weight[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


# If you need the version with "fasttext2", just instantiate the above classes with different embeddings:
# mean_vectorizer = MeanEmbeddingVectorizer(fasttext2)
# tfidf_vectorizer = TfidfEmbeddingVectorizer(fasttext2)
 

In [20]:
from mlxtend.classifier import StackingClassifier
from sklearn.pipeline import make_pipeline
#from mlxtend.feature_selection import ColumnSelector
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from mlxtend.preprocessing import DenseTransformer 
    
from sklearn.base import BaseEstimator
import numpy as np


class ColumnSelector(BaseEstimator):
    """Object for selecting specific columns from a data set.
    Parameters
    ----------
    cols : array-like (default: None)
        A list specifying the feature indices to be selected. For example,
        [1, 4, 5] to select the 2nd, 5th, and 6th feature columns, and
        ['A','C','D'] to select the name of feature columns A, C and D.
        If None, returns all columns in the array.
    drop_axis : bool (default=False)
        Drops last axis if True and the only one column is selected. This
        is useful, e.g., when the ColumnSelector is used for selecting
        only one column and the resulting array should be fed to e.g.,
        a scikit-learn column selector. E.g., instead of returning an
        array with shape (n_samples, 1), drop_axis=True will return an
        aray with shape (n_samples,).
    Examples
    -----------
    For usage examples, please see
    http://rasbt.github.io/mlxtend/user_guide/feature_selection/ColumnSelector/
    """

    def __init__(self, cols=None, drop_axis=False):
        self.cols = cols
        self.drop_axis = drop_axis

    def transform(self, X, y=None):
    """ Return a slice of the input array. ... [rest of the docstring] ... """

    # We use the loc or iloc accessor if the input is a pandas dataframe
    if hasattr(X, 'loc') or hasattr(X, 'iloc'):
        # Ensure cols is a list
        if isinstance(self.cols, tuple):
            self.cols = list(self.cols)
        
        # Check all elements in `cols` are of the same data type
        types = {type(i) for i in self.cols}
        if len(types) > 1:
            raise ValueError('Elements in `cols` should be all of the same data type.')
        
        if isinstance(self.cols[0], int):
            t = X.iloc[:, self.cols].values
        elif isinstance(self.cols[0], str):
            t = X.loc[:, self.cols].values
        else:
            raise ValueError('Elements in `cols` should be either `int` or `str`.')
    else:
        t = X[:, self.cols]

    if t.shape[-1] == 1 and self.drop_axis:
        t = t.reshape(-1)
    elif len(t.shape) == 1 and not self.drop_axis:
        t = t[:, np.newaxis]
    
    return t

    from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

def create_pipeline(column, vectorizer, classifier):
    return Pipeline([
        ("col_sel", ColumnSelector(cols=column, drop_axis=True)),
        ("vectorizer", vectorizer),
        ("classifier", classifier)
    ])

# Define classifiers
logistic_l2 = LogisticRegression("l2", random_state=0)
svc_linear = SVC(random_state=0, kernel="linear", tol=1e-5, probability=True)

# Construct pipelines
log_reg_fasttext_tfidf = create_pipeline(5, TfidfEmbeddingVectorizer(fasttext), logistic_l2)
log_reg_fasttext2 = create_pipeline(5, MeanEmbeddingVectorizer(fasttext), logistic_l2)
svm_fasttext = create_pipeline(5, MeanEmbeddingVectorizer(fasttext), svc_linear)
log_reg_fasttext_tfidf2 = create_pipeline(7, TfidfEmbeddingVectorizer(fasttext2), logistic_l2)
svm_fasttext_tfidf = create_pipeline(5, TfidfEmbeddingVectorizer(fasttext), svc_linear)

# Add any more pipelines as needed


In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from mlxtend.classifier import SuperLearner

RANDOM_STATE = 0
TRAIN_SIZE = 0.9  # 90% training, adjust as needed
COL_INDEX = 7     # adjust as per your specific requirements

def build_classifier_pipelines():
    """Create and return classifier pipelines."""
    
    log_reg_fasttext_tfidf = Pipeline([
        ("col_sel", ColumnSelector(cols=COL_INDEX, drop_axis=True)),
        ("fasttext vectorizer", TfidfEmbeddingVectorizer(fasttext)),
        ("log_reg", LogisticRegression("l2", random_state=RANDOM_STATE))
    ])
    
    svm_fasttext = Pipeline([
        ("col_sel", ColumnSelector(cols=COL_INDEX, drop_axis=True)),
        ("fasttext vectorizer", MeanEmbeddingVectorizer(fasttext)),
        ("LinearSVC", SVC(random_state=RANDOM_STATE, kernel="linear", tol=1e-5, probability=True))
    ])
    
    # ... [other pipelines]
    
    return [svm_fasttext, svm_fasttext_tfidf, log_reg_fasttext_tfidf2]

def train_ensemble(X_train, y_train):
    """Train ensemble using given training data."""
    
    ensemble = SuperLearner(scorer=accuracy_score, random_state=RANDOM_STATE)
    ensemble.add(build_classifier_pipelines())
    ensemble.add_meta(LogisticRegression("l2", random_state=RANDOM_STATE))
    
    ensemble.fit(X_train, y_train)
    
    return ensemble

def main():
    # Assuming `sample_df` and `sample_df.label` are your features and labels
    X_train, X_test, y_train, y_test = train_test_split(
        sample_df.values, sample_df.label.values, 
        train_size=TRAIN_SIZE, random_state=RANDOM_STATE
    )
    
    ensemble_model = train_ensemble(X_train, y_train)
    preds = ensemble_model.predict(X_test)
    
    accuracy = accuracy_score(y_test, preds)
    print(f"Accuracy: {accuracy:.4f}")

if __name__ == "__main__":
    main()


/Users/aikaterinikatsarou/homebrew_high_sierra/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7555012224938875


In [1]:
sample_df.head()

NameError: name 'sample_df' is not defined